# Summer Event Verification

## 1 - Import Necessary Modules

In [1]:
import cartopy 
import cartopy.io.shapereader as shpreader 
import cartopy.io.shapereader as Reader 
import geopandas as gpd
import matplotlib.pyplot as plt 
import numpy as np 
import proplot
import rioxarray
import salem
import xarray as xr 

from cartopy.feature import ShapelyFeature 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes 
from netCDF4 import Dataset 
from shapely.geometry import mapping
from wrf import xy_to_ll 

# warnings
import warnings
warnings.filterwarnings('ignore')

## 2 - Station Information

In [2]:
station_detail_prec = {   '17130': [39.9727, 32.8637, 'Ankara Bölge', 1],
                     '17134': [39.8032, 32.8434, 'Ufuk Danışment'],
                     '17651' : [39.1525, 32.1283, 'Polatlı Tigem'],
                     '17664' : [40.4729, 32.6441, 'Kızılcahamam'],
                     '17679' : [40.1733, 31.332, 'Nallıhan'],
                     '17680' : [40.1608, 31.9172, 'Beypazarı'],
                     '17715' : [39.92, 33.2125, 'Elmadağ Barutsan Fabrikası'],
                     '17728' : [39.5834, 32.1624, 'Polatlı'],
                     '17729' : [39.5546, 33.1089, 'Bala'],
                     '17731' : [38.9539, 33.4218, 'Şereflikoçhisar'],
                     '17733' : [39.613, 32.672, 'Haymana Tarım'],
                     '17759' : [39.7414, 32.38, 'Temelli'],
                     '17987' : [40.0386, 33.2908, 'Yeşildere'],
                     '18045' : [40.03, 32.2345, 'Ayaş'],
                     '18046' : [39.9075, 32.8494, 'TBMM Bahçesi'],
                     '18072' : [40.0878, 32.8111, 'Bağlum'],
                     '18073' : [40.2725, 32.6897, 'Kurtboğazı Barajı'],
                     '18074' : [39.9194, 32.9944, 'Bayındır Barajı'],
                     '18075' : [40.4839, 32.4608, 'Çamlıdere Orman İşletme Müdürlüğü'],
                     '18076' : [40.0947, 33.4133, 'Kalecik DSİ'],
                     '18077' : [40.2111, 32.2472, 'Güdül Gençlik Spor'],
                     '18078' : [39.5578, 31.9019, 'Acıkır'],
                     '18240' : [40.1408, 33.1081, 'Akyurt'],
                     '18241' : [40.5983, 32.5033, 'Çamkoru'],
                     '18242' : [40.2867, 33.0108, 'Çubuk'],
                     '18243' : [40.0317, 32.8933, 'Pursaklar'],
                     '18244' : [39.9986, 32.5814, 'Sincan'],
                     '18250' : [39.9725, 32.8639, '9. Ankara Bölge'],
                     '18256' : [39.8572, 32.8219, 'Çaldağ'],
                     '18257' : [39.4356, 32.5153, 'Haymana'],
                     '18903' : [40.0761, 32.6075, 'Sarayköy Toprak']}

station_detail_temp = {'17127' : [40.0788, 32.5657, 'Akıncı Airport'],
                     '17128' : [40.124, 32.9992, 'Esenboğa Airport'],
                     '17129' : [39.9558, 32.6854, 'Etimesgut Airport'],
                     '17130': [39.9727, 32.8637, 'Ankara Bölge'],
                     '17131' : [39.9343, 32.7387, 'Güvercinlik Airport'],
                     '17134': [39.8032, 32.8434, 'Ufuk Danışment'],
                     '17137': [39.7985, 32.9716, 'Elmadağ Radar'],
                     '17651' : [39.1525, 32.1283, 'Polatlı Tigem'],
                     '17664' : [40.4729, 32.6441, 'Kızılcahamam'],
                     '17679' : [40.1733, 31.332, 'Nallıhan'],
                     '17680' : [40.1608, 31.9172, 'Beypazarı'],
                     '17715' : [39.92, 33.2125, 'Elmadağ Barutsan Fabrikası'],
                     '17728' : [39.5834, 32.1624, 'Polatlı'],
                     '17729' : [39.5546, 33.1089, 'Bala'],
                     '17731' : [38.9539, 33.4218, 'Şereflikoçhisar'],
                     '17733' : [39.613, 32.672, 'Haymana Tarım']}

## 2 - Open Datasets

In [3]:
#aws obs precipitatio
path_prec = r'\datasets\ankara_P_tümveri_0918.xlsx'
dsp = pd.read_excel(path_prec, sheet_name='ankara_1318_p')

In [4]:
#aws obs temperature
path_temp = r'\datasets\ankara_0918T.xlsx'
dst = pd.read_excel(path_temp, sheet_name='ankara_0918t', na_values=-999)

In [5]:
#SUMMER
#thompson
wrf1_data = salem.open_wrf_dataset(r'\summer_datasets\ankara_thompson_urban\wrfout_d03_2016-08-26_00_00_00') 
wrf1_data_addit = Dataset(r'\summer_datasets\ankara_thompson_urban\wrfout_d03_2016-08-26_00_00_00')
#lin
wrf2_data = salem.open_wrf_dataset(r'\summer_datasets\ankara_lin_urban\wrfout_d03_2016-08-26_00_00_00')
wrf2_data_addit = Dataset(r'\summer_datasets\ankara_lin_urban\wrfout_d03_2016-08-26_00_00_00')
#wsm6
wrf3_data = salem.open_wrf_dataset(r'\summer_datasets\ankara_wsm6_urban\wrfout_d03_2016-08-26_00_00_00')
wrf3_data_addit = Dataset(r'\summer_datasets\ankara_wsm6_urban\wrfout_d03_2016-08-26_00_00_00')

## 3 - Extract desired hourly AWS observation Dataframe

In [6]:
def extract_hourly_point_obs_dataframe(data, start, end):
    """
    data: Pandas Data
    start: desired start date (e.g., '2009-01-01 00')
    end: desired end date
    """
    data_copy = data.copy(deep=True)
    data_start = str(data['year'].iloc[0]) + str(data['month'].iloc[0]) + str(data['day'].iloc[0]) + str(data['hour'].iloc[0])
    data_end = str(data['year'].iloc[-1]) + str(data['month'].iloc[-1]) + str(data['day'].iloc[-1]) + str(data['hour'].iloc[-1])
    data_start = pd.to_datetime(int(data_start), format='%Y%m%d%H')
    data_end = pd.to_datetime(int(data_end), format='%Y%m%d%H')

    data_copy.index = pd.date_range(data_start, data_end, freq='1H') #set datetime index
    del data_copy['year'], data_copy['month'], data_copy['day'], data_copy['hour']    
    data_copy = data_copy.loc[start : end]
    return data_copy

In [15]:
#extract point AWS precipitation dataset for the date of interest
obs_aws_prec = extract_hourly_point_obs_dataframe(dsp, '2016-08-28 00', '2016-08-28 21')
#optional
del obs_aws_prec[17725] #Has NaN values ; useless
del obs_aws_prec[17137] #Elmadağ Radar outlier in WRF

In [16]:
#extract point AWS temperature dataset for the date of interest
obs_aws_temp = extract_hourly_point_obs_dataframe(dst, '2016-08-28 00', '2016-08-28 21')
#if any station contains NaN values you may think to not use that station and delete it
#in my case there are no NaN values for the date of my interest

## 4 - Extract desired hourly point-wise WRF data

In [11]:
def extract_hourly_point_wrf_dataframe(data, data_addit, station_details, var, start=False, end=False):
    """
    data: WRF dataset opened with Salem 
    data_addit: WRF dataset opened with NetCDF4
    station_details: Dictionary for the details of 
                     the observation number and lat-lons (e.g., {'17130': [lat,lon]})
    var: variable name
    """
    #get the essential information
    data_updated = data[var]
    data_start = str(data[var].time[0].values)
    data_end = str(data[var].time[-1].values)
    
    #if start and end is given by user
    if start and end: 
        data_start = start
        data_end = end
        data_updated = data[var].sel(time=slice(start, end)) #update the dataset

    data_station_locs = [] #list to create pandas dataFrame
    for i in station_details:
        x,y = ll_to_xy(data_addit, station_details[i][0], station_details[i][1]) #get x-y values given the station's coordinates
        data_station_locs.append(data_updated.isel(south_north=y, west_east=x).values)
 
    data_station_locs = pd.DataFrame(np.array(data_station_locs).transpose(), columns=[int(i) for i in station_details])
    data_station_locs.index = pd.date_range(data_start, data_end, freq='1H') #set datetime index
    return data_station_locs

In [13]:
#Extract WRF point precipitation pandas data for the day of interest
wrf1_pandas_prec = extract_hourly_point_wrf_dataframe(wrf1_data, wrf1_data_addit, station_detail_prec, 'PRCP',
                                                      start='2016-08-28T00', end='2016-08-28T21') #21 UTC for my case, 23UTC for you Kutay
wrf2_pandas_prec = extract_hourly_point_wrf_dataframe(wrf2_data, wrf2_data_addit, station_detail_prec, 'PRCP',
                                                      start='2016-08-28T00', end='2016-08-28T21')
wrf3_pandas_prec = extract_hourly_point_wrf_dataframe(wrf3_data, wrf3_data_addit, station_detail_prec, 'PRCP',
                                                      start='2016-08-28T00', end='2016-08-28T21')

In [14]:
#Extract WRF point temperature pandas data for the day of interest
wrf1_pandas_temp = extract_hourly_point_wrf_dataframe(wrf1_data, wrf1_data_addit, station_detail_temp, 'T2',
                                                      start='2016-08-28T00', end='2016-08-28T21')
wrf2_pandas_temp = extract_hourly_point_wrf_dataframe(wrf2_data, wrf2_data_addit, station_detail_temp, 'T2',
                                                      start='2016-08-28T00', end='2016-08-28T21')
wrf3_pandas_temp = extract_hourly_point_wrf_dataframe(wrf3_data, wrf3_data_addit, station_detail_temp, 'T2',
                                                      start='2016-08-28T00', end='2016-08-28T21')

wrf1_pandas_temp = wrf1_pandas_temp - 273.15
wrf2_pandas_temp = wrf2_pandas_temp - 273.15
wrf3_pandas_temp = wrf3_pandas_temp - 273.15

## 5 - Calculate Verification Scores

In [34]:
class metricer(object):
    def __init__(self, actual, predicted):
        self.actual = actual
        self.predicted = predicted
    def __repr__(self):
        return "Shape of the actual observation array{} ; Shape of the predicted array{} ".format(np.shape(self.actual), np.shape(self.predicted))
    def calc_MAE(self):
        """
        Calculate Mean Absolute Error
        """
        return np.mean(np.abs(self.predicted - self.actual))
    
    def calc_RMSE(self):
        """
        Calculate Root Mean Squared Error
        """
        return np.sqrt(np.mean(np.square(self.predicted - self.actual)))
    
    def calc_Pearson_Corr(self):
        """
        Calculate Pearson Correlation Coef
        """
        return np.corrcoef(self.predicted, self.actual)

### 5.1 - Precipitation Metrics

In [35]:
#create metric object for each WRF run
metric_obj_thompson_prec = metricer(obs_aws_prec, wrf1_pandas_prec)
metric_obj_lin_prec = metricer(obs_aws_prec, wrf2_pandas_prec)
metric_obj_wsm6_prec = metricer(obs_aws_prec, wrf3_pandas_prec)

In [36]:
print(metric_obj_thompson_prec.calc_MAE().mean())
print(metric_obj_lin_prec.calc_MAE().mean())
print(metric_obj_wsm6_prec.calc_MAE().mean())

0.2219169539954056
0.2535415275406369
0.24467683790597208


In [37]:
print(metric_obj_thompson_prec.calc_RMSE().mean())
print(metric_obj_lin_prec.calc_RMSE().mean())
print(metric_obj_wsm6_prec.calc_RMSE().mean())

0.7354611273035044
0.8770822607785106
0.8200154676093309


### 5.2 - Temperature Metrics

In [38]:
#create metric object for each WRF run
metric_obj_thompson_temp = metricer(obs_aws_temp, wrf1_pandas_temp)
metric_obj_lin_temp = metricer(obs_aws_temp, wrf2_pandas_temp)
metric_obj_wsm6_temp = metricer(obs_aws_temp, wrf3_pandas_temp)

In [39]:
#Mean Absolute Error MAE
print(metric_obj_thompson_temp.calc_MAE().mean())
print(metric_obj_lin_temp.calc_MAE().mean())
print(metric_obj_wsm6_temp.calc_MAE().mean())

1.667840914879445
1.7140281461697011
1.7108141066246354


In [40]:
#Root Mean Square Error RMSE
print(metric_obj_thompson_temp.calc_RMSE().mean())
print(metric_obj_lin_temp.calc_RMSE().mean())
print(metric_obj_wsm6_temp.calc_RMSE().mean())

2.059785938684646
2.126212919985314
2.1212443715086358
